In [109]:
from bs4 import BeautifulSoup
import urllib
import re
import json

**The scripts for the second season are in different formats.**

This section handles parsing of Season 1, episodes 1-5 and all of Season 2.

In [110]:
lynchsite = urllib.urlopen('http://www.glastonberrygrove.net/texts/main.html').read()
lynchsite_soup = BeautifulSoup(lynchsite, 'html.parser')

In [111]:
script_links=['http://www.lynchnet.com/tp/tp01.html', 'http://www.lynchnet.com/tp/tp02.html', 'http://www.lynchnet.com/tp/tp03.html', 'http://www.lynchnet.com/tp/tp04.html', 'http://www.lynchnet.com/tp/tp05.html']
hand_script_links=[]
for link in lynchsite_soup.find_all('a'):
     if re.search('script\d',str(link.get('href'))) is not None:
            script_links.append('http://www.glastonberrygrove.net/texts/'+str(link.get('href')))

script_links

['http://www.lynchnet.com/tp/tp01.html',
 'http://www.lynchnet.com/tp/tp02.html',
 'http://www.lynchnet.com/tp/tp03.html',
 'http://www.lynchnet.com/tp/tp04.html',
 'http://www.lynchnet.com/tp/tp05.html',
 'http://www.glastonberrygrove.net/texts/script8.html',
 'http://www.glastonberrygrove.net/texts/script9.html',
 'http://www.glastonberrygrove.net/texts/script10.html',
 'http://www.glastonberrygrove.net/texts/script11.html',
 'http://www.glastonberrygrove.net/texts/script12.html',
 'http://www.glastonberrygrove.net/texts/script13.html',
 'http://www.glastonberrygrove.net/texts/script14.html',
 'http://www.glastonberrygrove.net/texts/script15.html',
 'http://www.glastonberrygrove.net/texts/script16.html',
 'http://www.glastonberrygrove.net/texts/script17.html',
 'http://www.glastonberrygrove.net/texts/script18.html',
 'http://www.glastonberrygrove.net/texts/script19.html',
 'http://www.glastonberrygrove.net/texts/script20.html',
 'http://www.glastonberrygrove.net/texts/script21.html',

Idea.... if I have a scene with no discussion, look for characters names?

Still missing data for Pilot, episode 6 and 7.

In [112]:
wiki_info = urllib.urlopen('https://en.wikipedia.org/wiki/List_of_Twin_Peaks_episodes').read()
wiki_soup = BeautifulSoup(wiki_info, 'html.parser')

In [113]:
description_array=[]
for a in wiki_soup.find_all("td", "description"):
    description_array.append(a.get_text())
    
print description_array



[u"In the feature-length pilot episode, the small town of Twin Peaks, Washington, is shaken up when the body of high school student Laura Palmer is discovered near a riverbank, wrapped in plastic. FBI Agent Dale Cooper is called by local sheriff Harry Truman when Ronette Pulaski, a schoolmate of Laura, is found wandering on a bridge and then slips into a coma. Cooper sees a connection between Laura's death and the murder of another girl one year earlier. After finding a piece of paper under Laura's fingernail, similar to one he found in the fingernail of the girl who died earlier, Cooper suspects that same killer might have struck again. Meanwhile, Sheriff Truman arrests Laura's boyfriend, Bobby Briggs, who is secretly having an affair with a married woman named Shelly Johnson. Afterwards, Laura's mother has a terrifying nightmare.", u"Cooper's investigation into the murder of Laura Palmer continues, as her secret boyfriend James Hurley is interrogated, and it appears that both trucker

In [114]:
episode_title_soup = []
episode_title_array = []
for a in wiki_soup.find_all("td", "summary"):
    episode_title_soup.append(a.get_text())
for episode in episode_title_soup:
    title = re.search('\"(.*)\".*\n',episode).group(1)
    subtitle = re.search('\n\"(.*)\"', episode).group(1)
    episode_title_array.append({'episode_title': title, 'episode_subtitle':subtitle})

episode_title_array

[{'episode_subtitle': u'Northwest Passage', 'episode_title': u'Pilot'},
 {'episode_subtitle': u'Traces to Nowhere', 'episode_title': u'Episode 1'},
 {'episode_subtitle': u'Zen, or the Skill to Catch a Killer',
  'episode_title': u'Episode 2'},
 {'episode_subtitle': u'Rest in Pain', 'episode_title': u'Episode 3'},
 {'episode_subtitle': u'The One-Armed Man', 'episode_title': u'Episode 4'},
 {'episode_subtitle': u"Cooper's Dreams", 'episode_title': u'Episode 5'},
 {'episode_subtitle': u'Realization Time', 'episode_title': u'Episode 6'},
 {'episode_subtitle': u'The Last Evening', 'episode_title': u'Episode 7'},
 {'episode_subtitle': u'May the Giant Be with You',
  'episode_title': u'Episode 8'},
 {'episode_subtitle': u'Coma', 'episode_title': u'Episode 9'},
 {'episode_subtitle': u'The Man Behind Glass', 'episode_title': u'Episode 10'},
 {'episode_subtitle': u"Laura's Secret Diary", 'episode_title': u'Episode 11'},
 {'episode_subtitle': u"The Orchid's Curse", 'episode_title': u'Episode 12'}

In [115]:
count = 0
episode_metadata = episode_title_array
for num in description_array:
    
    episode_metadata[count]['episode_description'] = num
    count +=1

episode_metadata
    
    

[{'episode_description': u"In the feature-length pilot episode, the small town of Twin Peaks, Washington, is shaken up when the body of high school student Laura Palmer is discovered near a riverbank, wrapped in plastic. FBI Agent Dale Cooper is called by local sheriff Harry Truman when Ronette Pulaski, a schoolmate of Laura, is found wandering on a bridge and then slips into a coma. Cooper sees a connection between Laura's death and the murder of another girl one year earlier. After finding a piece of paper under Laura's fingernail, similar to one he found in the fingernail of the girl who died earlier, Cooper suspects that same killer might have struck again. Meanwhile, Sheriff Truman arrests Laura's boyfriend, Bobby Briggs, who is secretly having an affair with a married woman named Shelly Johnson. Afterwards, Laura's mother has a terrifying nightmare.",
  'episode_subtitle': u'Northwest Passage',
  'episode_title': u'Pilot'},
 {'episode_description': u"Cooper's investigation into t

In [116]:
def script_script(url):
    r = urllib.urlopen(url).read()
    soup = BeautifulSoup(r, 'html.parser')
    script = soup.get_text()
    #gets all the scenes and characters
    scenes_chars = re.findall('\n\s*(\d.+|[A-Z ]+\s*)\n\s*', script)
    scene = 0
    scene_array = []
    #parses the script into a dictionary
    for line in scenes_chars:
        scene_num = re.search('(\d+\w*)[\.\: ]', line)
        #establishes the scene and all its info
        if scene_num is not None:
            #print line
            scene += 1
            scene_array.append({'scene_number': int(scene), 'characters':[], 'heading':{}})
            #inside or outside
            if "INT" in line:
                scene_array[scene-1]['heading']['int_ext'] = 'INT'
            elif "EXT" in line:
                scene_array[scene-1]['heading']['int_ext'] = 'EXT'
            #grabs location
            location = re.search('[A-Z]\.\s+(.*)\s+\-',line)
            if location is not None:
                scene_array[scene-1]['heading']['location'] = location.group(1)
            #gets time
            time = re.search('- ([A-Z]+)$',line)
            if time is not None:
                scene_array[scene-1]['heading']['time'] = time.group(1)
        elif scene == 0:
            pass
        else:
            if 'FADE' not in line and 'ACT' not in line and 'SCENE' not in line:
                if line not in scene_array[scene-1]['characters'] and line !=" " and "\n" not in line:
                    scene_array[scene-1]['characters'].append(line)
    #print scene_array
    return scene_array


In [120]:
#This puts all of the data together and exports it to json
episode_array = []
for link in script_links:
    episode_num = int(re.search('(\d*).html',link).group(1))
    #this adds the episode metadata and scene information
    episode_array.append({'episode_number': episode_num, 'episode_description': episode_metadata[episode_num]['episode_description'], 'episode_title': episode_metadata[episode_num]['episode_title'], 'episode_subtitle': episode_metadata[episode_num]['episode_subtitle'], 'scenes': script_script3(link)})
print episode_array
outfile = open('twin_peaks.json', 'w')
json.dump(episode_array, outfile, indent=4)
outfile.close()

Cooper's investigation into the murder of Laura Palmer continues, as her secret boyfriend James Hurley is interrogated, and it appears that both trucker Leo Johnson and Laura's psychiatrist Dr. Jacoby may have some connection to the crime. Laura's best friend Donna and Audrey Horne vow to solve the murder.
[{'episode_title': u'Episode 1', 'episode_subtitle': u'Traces to Nowhere', 'episode_number': 1, 'scenes': [{'scene_number': 1, 'heading': {'int_ext': 'EXT', 'location': u'GREAT NORTHERN HOTEL', 'time': u'DAY'}, 'characters': []}, {'scene_number': 2, 'heading': {'int_ext': 'INT', 'location': u'GREAT NORTHERN HOTEL ROOM', 'time': u'DAY'}, 'characters': [u'COOPER']}, {'scene_number': 3, 'heading': {'int_ext': 'EXT', 'location': u"SHERIFF TRUMAN'S HOUSE", 'time': u'MORNING'}, 'characters': [u'TRUMAN']}, {'scene_number': 4, 'heading': {'int_ext': 'EXT', 'location': u'DOUGHNUT SHOP', 'time': u'DAY'}, 'characters': []}, {'scene_number': 5, 'heading': {'int_ext': 'INT', 'location': u'DOUGHNU